In [1]:
import numpy as np

def exact_mean_spin(rbm):
    """
    Compute the exact expectation value <s_i> for each visible spin
    using full enumeration of all visible configurations.
    """
    n_vis = rbm.n_visible
    # All configurations of spins ∈ {-1, +1}^n_vis
    all_states = np.array(np.meshgrid(*[[-1, 1]] * n_vis)).T.reshape(-1, n_vis)

    # Unnormalized probabilities from the RBM wavefunction
    probs = np.real(rbm.prob(all_states))
    probs /= probs.sum()  # normalize to sum to 1

    # Expectation value of each visible spin
    mean_spins = np.sum(all_states * probs[:, None], axis=0)
    return mean_spins, all_states, probs


In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator

# sim = AerSimulator(device="GPU")

# ------------------- your ComplexRBMCustom (unchanged) -------------------
# (use your existing class; shortened here)
class ComplexRBMCustom:
    def __init__(self, n_visible, n_hidden, edges, seed=None):
        rng = np.random.default_rng(seed)
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.edges = edges
        self.a = 0.2 * (rng.standard_normal(n_visible) + 1j * rng.standard_normal(n_visible))
        self.b = 0.2 * (rng.standard_normal(n_hidden) + 1j * rng.standard_normal(n_hidden))
        self.W = {(v,h): 0.3 * (rng.standard_normal() + 1j * rng.standard_normal()) for (v,h) in edges}

    def log_psi(self, x):
        x = np.asarray(x)
        if x.ndim == 1:
            x = x[None,:]
        Wx = np.zeros((x.shape[0], self.n_hidden), dtype=np.complex128)
        for (v,h), w in self.W.items():
            Wx[:,h] += w * x[:,v]
        Wx += self.b
        return (x @ self.a).sum(axis=-1) + np.sum(np.log(2*np.cosh(Wx)), axis=-1)

    def psi(self, x):
        return np.exp(self.log_psi(x))

    def prob(self, x):
        x = np.asarray(x)
        if x.ndim == 1:
            return float(np.abs(self.psi(x))**2)
        else:
            return np.abs(self.psi(x))**2

# ------------------- helpers: build QAOA (same logic you used) -------------------
def apply_ZZ_phase(circ, q_i, q_j, angle):
    circ.cx(q_i, q_j)
    circ.rz(2*angle, q_j)
    circ.cx(q_i, q_j)

def apply_Z_phase(circ, q, angle):
    circ.rz(2*angle, q)

def build_qaoa_initialized(rbm, edges, num_layers, gammas, betas, c_i):
    n_vis = rbm.n_visible
    n_hid = rbm.n_hidden
    n_qubits = n_vis + n_hid
    qreg = QuantumRegister(n_qubits, "q")
    creg_vis = ClassicalRegister(n_vis, "c_vis")
    qc = QuantumCircuit(qreg, creg_vis)

    # init visible to c_i (spin +1 -> bit 0, -1 -> bit 1)
    bits_i = np.where(np.asarray(c_i)==1, 0, 1)
    for i,b in enumerate(bits_i):
        if int(b)==1:
            qc.x(qreg[i])
    # init hidden to |+>
    for j in range(n_hid):
        qc.h(qreg[n_vis + j])

    a_real = np.real(rbm.a); b_real = np.real(rbm.b)
    for layer in range(num_layers):
        gamma = gammas[layer]; beta = betas[layer]
        for i in range(n_vis):
            coeff = -gamma * a_real[i]
            if coeff != 0.0:
                apply_Z_phase(qc, qreg[i], coeff)
        for j in range(n_hid):
            coeff = -gamma * b_real[j]
            if coeff != 0.0:
                apply_Z_phase(qc, qreg[n_vis+j], coeff)
        for (v,h) in edges:
            w_r = float(np.real(rbm.W[(v,h)]))
            if w_r != 0:
                apply_ZZ_phase(qc, qreg[v], qreg[n_vis+h], -gamma * w_r)
        for q in range(n_qubits):
            qc.rx(2*beta, qreg[q])
    for i in range(n_vis):
        qc.measure(qreg[i], creg_vis[i])
    return qc, n_vis

def canonicalize_bitstring(bitstring, n_vis):
    s = bitstring.replace(" ", "")
    if len(s) >= n_vis:
        s = s[-n_vis:]
    return s[::-1]   # reversed so index 0 -> c_vis[0]

# ------------------- QuantumProposer object -------------------
class QuantumProposer:
    def __init__(self, rbm, edges, num_layers, gammas, betas, shots=256):
        self.rbm = rbm
        self.edges = edges
        self.num_layers = num_layers
        self.gammas = gammas
        self.betas = betas
        self.shots = shots
        self.sim_base = AerSimulator()  # we will set seed per-run
        # caching optional: small cache to avoid re-running same initialization many times
        self._cache = {}

    def _run_qaoa(self, c_init, seed_sim):
        """Run QAOA initialized at c_init and return empirical dict {canonical_bitstr: prob}."""
        key = (tuple(c_init.tolist()), seed_sim)
        if key in self._cache:
            return self._cache[key]
        qc, n_vis = build_qaoa_initialized(self.rbm, self.edges, self.num_layers, self.gammas, self.betas, c_init)
        # sim = AerSimulator(method="statevector", device="GPU", seed_simulator=int(seed_sim) if seed_sim is not None else None)
        # sim = AerSimulator(device="GPU")
        tqc = transpile(qc, self.sim_base)
        job = self.sim_base.run(tqc, shots=self.shots)
        res = job.result()
        counts = res.get_counts()
        total = sum(counts.values()) if counts else 0
        probs = {}
        for k,v in counts.items():
            kb = canonicalize_bitstring(k, n_vis)
            probs[kb] = probs.get(kb,0) + v
        if total>0:
            for k in probs:
                probs[k] = probs[k] / total
        else:
            # fallback deterministic init state
            bits = np.where(np.asarray(c_init)==1, 0, 1).astype(int)
            keystr = ''.join(str(b) for b in bits.tolist())
            probs = {keystr: 1.0}
        self._cache[key] = probs
        return probs

    def propose_with_prob(self, c_i, rng):
        """
        Build empirical distribution q(·|c_i) from shots, sample one c_f, return:
          (c_f, q_forward, probs_dict)
        """
        seed_sim = rng.integers(0,2**30)
        probs = self._run_qaoa(c_i, seed_sim)
        keys = list(probs.keys())
        pvals = np.array([probs[k] for k in keys], dtype=float)
        if pvals.sum() == 0:
            pvals = np.ones_like(pvals) / len(pvals)
        else:
            pvals = pvals / pvals.sum()
        idx = rng.choice(len(keys), p=pvals)
        chosen = keys[idx]
        bits = [int(ch) for ch in chosen]   # canonical has bit0 -> c_vis[0]
        spins = np.array([+1 if b==0 else -1 for b in bits], dtype=int)
        return spins, probs[chosen], probs

    def q_prob(self, c_f, c_i, rng):
        """Return empirical q(c_f|c_i) by running QAOA with init c_i and looking up c_f."""
        seed_sim = rng.integers(0,2**30)
        probs = self._run_qaoa(c_i, seed_sim)
        bits = ''.join(('0' if s==1 else '1') for s in c_f)  # canonical key expects bit0->c_vis[0]
        return float(probs.get(bits, 0.0))

# ------------------- Sampler that uses proposer which returns probs -------------------
class Sampler:
    def __init__(self, rbm, proposer, seed=None):
        """
        proposer can be:
          - classical: function (x, rng) -> new_x  (treated symmetric q ratio = 1)
          - quantum-like: object with propose_with_prob(x, rng) and q_prob(c_f,c_i,rng)
        """
        self.rbm = rbm
        self.proposer = proposer
        self.rng = np.random.default_rng(seed)

    def metropolis_step(self, x):
        """Perform one Metropolis-Hastings step starting from configuration x."""
        x = np.copy(x)
        # default: reject
        acc_prob = 0.0
        # detect proposer type (quantum-style proposer exposes propose_with_prob)
        if hasattr(self.proposer, "propose_with_prob"):
            # quantum: need forward prob and backward prob
            x_new, q_forward, _ = self.proposer.propose_with_prob(x, self.rng)
            # proposer may return None (no proposal) or zero forward prob -> reject
            if x_new is None or q_forward == 0.0:
                return x, False
            # compute backward probability q(x | x_new)
            q_backward = self.proposer.q_prob(x, x_new, self.rng)
            if q_backward == 0.0:
                # no backward support -> reject
                acc_prob = 0.0
            else:
                p_old = float(self.rbm.prob(x))
                p_new = float(self.rbm.prob(x_new))
                # guard against zero-probability current state to avoid div-by-zero
                if p_old == 0.0:
                    acc_prob = 1.0 if p_new > 0.0 else 0.0
                else:
                    ratio = (p_new / p_old) * (q_backward / q_forward)
                    acc_prob = float(min(1.0, ratio))
        else:
            # classical symmetric proposer function(x, rng) -> new_x
            x_new = self.proposer(x, self.rng)
            if x_new is None:
                return x, False
            p_old = float(self.rbm.prob(x))
            p_new = float(self.rbm.prob(x_new))
            if p_old == 0.0:
                acc_prob = 1.0 if p_new > 0.0 else 0.0
            else:
                acc_prob = float(min(1.0, p_new / p_old))

        if self.rng.random() < acc_prob:
            return x_new, True
        return x, False

    def sample(self, x0, n_steps=1000, burn_in=100):
        x = np.copy(x0)
        samples = []
        accepts = 0
        for step in range(n_steps + burn_in):
            x, accepted = self.metropolis_step(x)
            if accepted:
                accepts += 1
            if step >= burn_in:
                samples.append(np.copy(x))
        samples = np.array(samples)
        acc_rate = accepts / (n_steps + burn_in)
        return samples, acc_rate

# ------------------- small ESS helper -------------------
def autocorr_1d(x, maxlag=None):
    x = np.asarray(x, dtype=float)
    n = x.size
    x = x - x.mean()
    if maxlag is None:
        maxlag = n//2
    acf = np.array([1.0] + [np.corrcoef(x[:-lag], x[lag:])[0,1] for lag in range(1, maxlag)])
    return acf

def ess_from_chain(chain, maxlag=100):
    # chain shape (n_samples, n_dims)
    if chain.ndim == 1:
        chain = chain[:,None]
    n = chain.shape[0]
    ess = []
    for d in range(chain.shape[1]):
        ac = autocorr_1d(chain[:,d], maxlag=min(maxlag, n//2))
        # sum positive part
        pos = ac[1:][ac[1:]>0]
        tau = 1 + 2*np.sum(pos)
        ess.append(n / tau)
    return np.array(ess)

# ------------------- Example usage -------------------
if __name__ == "__main__":
    edges = [(0,0),(1,0),(2,1), (3,1), (4,2), (5,2), (6,3), (7,3)]
    rbm = ComplexRBMCustom(8, 4, edges, seed=123)

    rbm.W = {
        (0,0): 0.5, (1,0): -0.5,
        (2,1): 0.4, (3,1): -0.04,
        (4,2): 0.03, (5,2): -0.3,
        (6,3): 0.02, (7,3): -0.02,
    }

    gammas = [0.8 for _ in range(10)]
    betas = [-0.3 for _ in range(10)]

    qprop = QuantumProposer(rbm, edges, num_layers=10, gammas=gammas, betas=betas, shots=1000)
    classical_sampler = Sampler(rbm, proposer=lambda x,rng: (x if False else None))  # placeholder

    # Classical single-spin proposer wrapper (symmetric)
    def single_spin_flip_wrapper(x, rng):
        x_new = x.copy()
        i = rng.integers(len(x))
        x_new[i] *= -1
        return x_new
    
    def random_wrapper(x, rng):
        return rng.choice([-1, +1], size=x.shape)

    classical = Sampler(rbm, proposer=single_spin_flip_wrapper)
    random = Sampler(rbm, proposer=random_wrapper)
    quantum = Sampler(rbm, proposer=qprop)

    x0 = np.ones(rbm.n_visible, dtype=int)
    n_steps = 100
    samples_classical, acc_classical = classical.sample(x0, n_steps=n_steps, burn_in=50)
    samples_random, acc_random = random.sample(x0, n_steps=n_steps, burn_in=50)
    samples_quantum, acc_quantum = quantum.sample(x0, n_steps=n_steps, burn_in=50)

    print("Classical acc:", acc_classical, "Quantum acc:", acc_quantum, "Random acc:", acc_random)
    print("Exact mean (compute separately), sampled means:")
    print("Classical mean:", samples_classical.mean(axis=0))
    print("Random mean:   ", samples_random.mean(axis=0))
    print("Quantum mean:  ", samples_quantum.mean(axis=0))

    # ESS diagnostics (per-visible spin)
    print("ESS classical:", ess_from_chain(samples_classical))
    print("ESS random:", ess_from_chain(samples_random))
    print("ESS quantum:", ess_from_chain(samples_quantum))

    # autocorr diagnostics (first 20 lags)
    # print("Autocorr classical (first 20 lags):")
    # for i in range(rbm.n_visible):
    #     acf = autocorr_1d(samples_classical[:,i], maxlag=20)
    #     print(f" Spin {i}: ", acf)
    # print("Autocorr random (first 20 lags):")
    # for i in range(rbm.n_visible):
    #     acf = autocorr_1d(samples_random[:,i], maxlag=20)
    #     print(f" Spin {i}: ", acf)
    # print("Autocorr quantum (first 20 lags):")
    # for i in range(rbm.n_visible):
    #     acf = autocorr_1d(samples_quantum[:,i], maxlag=20)
    #     print(f" Spin {i}: ", acf)


/tmp/ipykernel_645098/164449771.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(np.abs(self.psi(x))**2)


Classical acc: 0.6533333333333333 Quantum acc: 0.56 Random acc: 0.56
Exact mean (compute separately), sampled means:
Classical mean: [-0.62  0.38  0.36 -0.04  0.56 -0.04 -0.6   0.22]
Random mean:    [-0.2  -0.2   0.22  0.14  0.24  0.16 -0.5   0.32]
Quantum mean:   [ 0.06 -0.44  0.44  0.12  0.58  0.28  1.    1.  ]
ESS classical: [8.29452009 5.38228353 9.01049821 7.71836225 6.38867747 7.24339683
 4.92450367 5.89740099]
ESS random: [12.20309526 13.69121566 17.55249785 16.60550976 18.35615425  8.83275009
 12.1539281  19.84554884]
ESS quantum: [ 13.89388307  15.4279387   12.65907844   5.18662911   6.22404756
  23.5983737  100.         100.        ]


In [48]:
mean_exact, states, probs = exact_mean_spin(rbm)
print("Classical acc:", acc_classical, "Quantum acc:", acc_quantum, "Random acc:", acc_random)
print("Exact mean spins:", mean_exact)
print(f"Mean spins (classical): {samples_classical.mean(axis=0)}")
print(f"Mean spins (random):   {samples_random.mean(axis=0)}")
print(f"Mean spins (quantum):   {samples_quantum.mean(axis=0)}")
print(f"Error (classical): {np.mean((mean_exact - samples_classical.mean(axis=0))**2)}\n",
      f"Error (random):   {np.mean((mean_exact - samples_random.mean(axis=0))**2)}\n",
      f"Error (quantum):  {np.mean((mean_exact - samples_quantum.mean(axis=0))**2)}")


Classical acc: 0.6533333333333333 Quantum acc: 0.56 Random acc: 0.56
Exact mean spins: [ 0.33997594 -0.33997594 -0.10192799  0.01225367 -0.00442164  0.03962477
  0.0028398  -0.0028398 ]
Mean spins (classical): [-0.62  0.38  0.36 -0.04  0.56 -0.04 -0.6   0.22]
Mean spins (random):   [-0.2  -0.2   0.22  0.14  0.24  0.16 -0.5   0.32]
Mean spins (quantum):   [ 0.06 -0.44  0.44  0.12  0.58  0.28  1.    1.  ]
Error (classical): 0.2992515456628225
 Error (random):   0.10780369657916322
 Error (quantum):  0.3491289484672167
